In [15]:
#@title **Importar bibliotecas necessárias** { vertical-output: true, form-width: "60%" }

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import gdown

## Importar amostras de 100.000 em cada dataset por link compartilhado

In [16]:
# Amostra SRAG
srag_file_id = '1E1N10m0YPm0g2PvAnSO5YYc1sFfqrzRi'
srag_url = f'https://drive.google.com/uc?id={srag_file_id}'
srag_output = 'SRAG-2021-RESIZED.csv'
gdown.download(srag_url, srag_output, quiet=False)
df_srag = pd.read_csv(srag_output, sep=',')

# Amostra SIM
sim_file_id = '1P1-TtXlJMgyvVPStpAWHUES55ZqSMoq2'
sim_url = f'https://drive.google.com/uc?id={sim_file_id}'
sim_output = 'SIM-2021-RESIZED.csv'
gdown.download(sim_url, sim_output, quiet=False)
df_sim = pd.read_csv(sim_output, sep=',')


Downloading...
From: https://drive.google.com/uc?id=1E1N10m0YPm0g2PvAnSO5YYc1sFfqrzRi
To: /content/SRAG-2021-RESIZED.csv
100%|██████████| 56.7M/56.7M [00:00<00:00, 203MB/s]
/tmp/ipython-input-16-738819002.py:6: DtypeWarning: Columns (59,61,62,63,91,105,114,116,117,118,122,143) have mixed types. Specify dtype option on import or set low_memory=False.
  df_srag = pd.read_csv(srag_output, sep=',')
Downloading...
From: https://drive.google.com/uc?id=1P1-TtXlJMgyvVPStpAWHUES55ZqSMoq2
To: /content/SIM-2021-RESIZED.csv
100%|██████████| 29.5M/29.5M [00:00<00:00, 94.6MB/s]
/tmp/ipython-input-16-738819002.py:13: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sim = pd.read_csv(sim_output, sep=',')


## Importar arquivos CSV (Caso a opção anterior não funcione)
- Links:

  - SRAG: https://drive.usercontent.google.com/uc?id=1E1N10m0YPm0g2PvAnSO5YYc1sFfqrzRi&export=download
  - SIM: https://drive.usercontent.google.com/uc?id=1P1-TtXlJMgyvVPStpAWHUES55ZqSMoq2&export=download

- Baixe os arquivos das amostras
- Crie uma pasta "Colab Notebooks" caso não exista e faça o uploado dos CSVs nela.

In [33]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

df_srag = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SRAG-2021-RESIZED.csv', sep=',')

df_sim = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SIM-2021-RESIZED.csv', sep=',')

Mounted at /content/drive/
